# Fine_tuning LLM for generating graph

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/onyxia/work/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.9: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.568 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.12.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep

In [4]:
import json
with open("fine_tuning_data.json", "r") as f:
    fine_tuning_data = json.load(f)
def tokenize_data(data, tokenizer):
    tokenized_dataset = []
    eos_token = tokenizer.eos_token  # Ensure the tokenizer has an EOS token
    print(len(data))
    for data_point in data:
        prompt = data_point["prompt"]
        output = str(data_point["output"])

        # Add the EOS token to signify the end of the sequence
        full_prompt = f"{prompt} {output} {eos_token}" if eos_token else f"{prompt} {output}"
        # print(full_prompt)
        # Tokenize the sequence
        tokenized_entry = tokenizer(
            full_prompt,
            padding=True,
            truncation=True,
            return_tensors="pt"
        )

        tokenized_dataset.append({
            "input_ids": tokenized_entry["input_ids"].squeeze(),
            "attention_mask": tokenized_entry["attention_mask"].squeeze()
        })
    return tokenized_dataset
tokenized_data = tokenize_data(fine_tuning_data, tokenizer)

8000


In [5]:
from torch.utils.data import Dataset

class FineTuningDataset(Dataset):
    def __init__(self, tokenized_data):
        self.data = tokenized_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            "input_ids": self.data[idx]["input_ids"],
            "attention_mask": self.data[idx]["attention_mask"],
        }


dataset = FineTuningDataset(tokenized_data)

<a name="Train"></a>
### Train the model

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [7]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.568 GB.
6.004 GB of memory reserved.


In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 8,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,0.390200
2,0.537200
3,0.336700
4,0.256400
5,0.310600
6,0.394200
7,0.329400
8,0.144800
9,0.299300
10,0.191500


In [9]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

5742.4351 seconds used for training.
95.71 minutes used for training.
Peak reserved memory = 9.99 GB.
Peak reserved memory for training = 3.986 GB.
Peak reserved memory % of max memory = 68.575 %.
Peak reserved memory for training % of max memory = 27.361 %.


In [ ]:
# import re
# import pandas as pd
# # for test dataset
# def extract_graph_data(line):
#     """
#     Extract graph features from a single line of text, accounting for variations in phrasing.
#     """
#     try:
#         graph_id = re.search(r"(graph_\d+)", line).group(1)
#         nodes_match = re.search(r"(\d+) nodes", line)
#         edges_match = re.search(r"(\d+) edges", line)
#         avg_degree_match = re.search(r"(average degree.*?([\d.]+)|each node is connected to ([\d.]+))", line)
#         triangles_match = re.search(r"(\d+) triangles", line)
#         clustering_coeff_match = re.search(r"clustering coefficient.*?([\d.]+)", line)
#         max_k_core_match = re.search(r"maximum k-core.*?(\d+)", line)
#         communities_match = re.search(r"(?:number of communities equal to (\d+)|(\d+) communities)", line)

#         print(f"Processing line: {line}")
#         print(f"Match for communities: {communities_match}")

#         nodes = int(nodes_match.group(1)) if nodes_match else None
#         edges = int(edges_match.group(1)) if edges_match else None
#         avg_degree = (
#             float(avg_degree_match.group(2) or avg_degree_match.group(3))
#             if avg_degree_match
#             else None
#         )
#         triangles = int(triangles_match.group(1)) if triangles_match else None
#         clustering_coeff = (
#             float(clustering_coeff_match.group(1).strip(".")) if clustering_coeff_match else None
#         )
#         max_k_core = int(max_k_core_match.group(1)) if max_k_core_match else None
#         communities = (
#             int(communities_match.group(1) or communities_match.group(2))
#             if communities_match
#             else None
#         )

#         return {
#             "graph_id": graph_id,
#             "nodes": nodes,
#             "edges": edges,
#             "avg_degree": avg_degree,
#             "triangles": triangles,
#             "clustering_coeff": clustering_coeff,
#             "max_k_core": max_k_core,
#             "communities": communities,
#         }
#     except AttributeError as e:
#         print(f"Error processing line: {line}")
#         print(f"Exception: {e}")
#         return None



# def process_file(file_path):
#     """
#     Process a text file containing graph descriptions.
#     """
#     data = []
#     with open(file_path, "r") as file:
#         for line in file:
#             line = line.strip() 
#             if line: 
#                 graph_data = extract_graph_data(line)
#                 if graph_data: 
#                     data.append(graph_data)
#     return pd.DataFrame(data)

# file_path = "data/test/test.txt" 
# output_csv = "graph_features.csv"

# df = process_file(file_path)
# if not df.empty:
#     print(df) 

#     df.to_csv(output_csv, index=False)
#     print(f"Data saved to {output_csv}")
# else:
#     print("No valid data extracted.")


Processing line: graph_0,This graph comprises 50 nodes and 589 edges. The average degree is equal to 23.56 and there are 3702 triangles in the graph. The global clustering coefficient and the graph's maximum k-core are 0.6226034308779012 and 18 respectively. The graph consists of 3 communities.
Match for communities: <re.Match object; span=(264, 277), match='3 communities'>
Processing line: graph_1,This graph comprises 20 nodes and 19 edges. The average degree is equal to 1.9 and there are 0 triangles in the graph. The global clustering coefficient and the graph's maximum k-core are 0 and 1 respectively. The graph consists of 4 communities.
Match for communities: <re.Match object; span=(240, 253), match='4 communities'>
Processing line: graph_2,This graph comprises 28 nodes and 165 edges. The average degree is equal to 11.785714285714286 and there are 387 triangles in the graph. The global clustering coefficient and the graph's maximum k-core are 0.4742647058823529 and 8 respectively. 

In [45]:
df

,graph_id,nodes,edges,avg_degree,triangles,clustering_coeff,max_k_core,communities
0,graph_0,50,589,23.560000,3702,0.622603,0,3
1,graph_1,20,19,1.900000,0,0.000000,0,4
2,graph_2,28,165,11.785714,387,0.474265,0,3
3,graph_3,47,1050,44.680851,14914,0.973626,0,1
4,graph_4,25,75,6.000000,22,0.176000,6,3
...,...,...,...,...,...,...,...,...
995,graph_995,17,17,2.000000,1,0.176471,2,4
996,graph_996,41,552,26.926829,3254,0.672221,0,4
997,graph_997,42,210,10.000000,129,0.204762,0,4
998,graph_998,17,33,3.882353,11,0.270492,3,3


In [ ]:
import json

def generate_prompts_to_json(df, output_file):
    """
    Generate prompts for each row in the DataFrame and save to a JSON file.
    
    Args:
        df (pd.DataFrame): DataFrame containing the graph attributes.
        output_file (str): Path to the output .json file.
    """
    prompts = {}
    for _, row in df.iterrows():
        graph_id = row['graph_id']
        prompt = (
            f"<human>: Generate a graph with {int(row['nodes'])} nodes, {int(row['edges'])} edges, "
            f"an average degree of {row['avg_degree']:.4f}, {int(row['triangles'])} triangles, "
            f"a clustering coefficient of {row['clustering_coeff']:.4f}, a maximum k-core of {int(row['max_k_core'])}, "
            f"and {int(row['communities'])} communities. "
            f"Output the edge list only in the format [(source, target), ...]. Ensure the graph respects the specified number of nodes, edges, and constraints. "
            f"Do not provide explanations or additional text.\n<assistant>:"
        )
        prompts[graph_id] = prompt

    with open(output_file, "w") as f:
        json.dump(prompts, f, indent=4)
    print(f"Prompts saved to {output_file}")

output_file = "graph_prompts.json"
generate_prompts_to_json(df, output_file)

Prompts saved to graph_prompts.json


<a name="Inference"></a>

In [ ]:
FastLanguageModel.for_inference(model)  # Optimize model for inference
# model = model.to("cuda")  # Move model to GPU

prompt ="<human>: Generate a graph with 50 nodes, 589 edges, an average degree of 23.5600, 3702 triangles, a clustering coefficient of 0.6226, a maximum k-core of 0, and 3 communities. Output the edge list only in the format [(source, target), ...]. Ensure the graph respects the specified number of nodes, edges, and constraints. Do not provide explanations or additional text.\n<assistant>:"

inputs = tokenizer(
    [prompt],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True)

decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(decoded_outputs)

['<human>: Generate a graph with the following properties:\n- 10 nodes and 5 edges\n- Average degree: 1.00\n- 0 triangles\n- Clustering coefficient: 0.00\n- Maximum k-core: 1\n- 5 communities\nOutput the edge list only in the format [(source, target),...]. Ensure the graph respects the specified number of nodes, edges, and constraints.\nDo not provide explanations or additional text.\n<assistant>: [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9]] ']
Generated Edge List:
[(source, target),...]


In [ ]:
import json
import csv
from tqdm import tqdm

def save_raw_model_output(json_file, output_csv, tokenizer, model, device="cuda", model_max_tokens=2048):
    """
    Save raw model outputs for all prompts in a JSON file and save them to a CSV file.

    Args:
        json_file (str): Path to the JSON file containing prompts.
        output_csv (str): Path to the output CSV file.
        tokenizer: Tokenizer for the model.
        model: Pretrained model for generation.
        device (str): Device to run the model (e.g., 'cuda' or 'cpu').
        model_max_tokens (int): Maximum token limit for the model.
    """
    FastLanguageModel.for_inference(model)
    # model = model.to(device) 

    def calculate_max_new_tokens(prompt, model_max_tokens, input_token_count):
        """
        Calculate max_new_tokens based on the number of edges in the prompt.

        Args:
            prompt (str): The text prompt for the model.
            model_max_tokens (int): The maximum token limit for the model.
            input_token_count (int): The number of tokens in the input.

        Returns:
            int: The calculated max_new_tokens.
        """
        try:
            # Extract the number of edges
            number_of_edges = int(prompt.split("edges,")[0].split()[-1])
            # Estimate the number of output tokens
            estimated_output_tokens = number_of_edges * 3 + 100  # ~3 tokens per edge + buffer
            return min(estimated_output_tokens, model_max_tokens - input_token_count)
        except ValueError:
            return max(300, model_max_tokens - input_token_count - 100)


    with open(json_file, "r") as f:
        prompts = json.load(f)

    with open(output_csv, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["graph_id", "raw_output"])

        for graph_id, prompt in tqdm(prompts.items(), desc="Processing prompts"):
            inputs = tokenizer([prompt], return_tensors="pt").to(device)
            input_token_count = inputs["input_ids"].size(1)

            max_new_tokens = calculate_max_new_tokens(prompt, model_max_tokens, input_token_count)

            print(f"Processing {graph_id}:")
            print(f"  Input tokens: {input_token_count}")
            print(f"  Max new tokens: {max_new_tokens}")

            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                use_cache=True
            )
            decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

            print(f"Raw Output for {graph_id}: {decoded_output}")

            writer.writerow([graph_id, decoded_output])

    print(f"Raw model outputs saved to {output_csv}")

json_file = "graph_prompts.json" 
output_csv = "raw_model_outputs.csv"

save_raw_model_output(json_file, output_csv, tokenizer, model)


Processing prompts:   0%|          | 0/1000 [00:00<?, ?it/s]

Processing graph_0:
  Input tokens: 95
  Max new tokens: 1867


Processing prompts:   0%|          | 1/1000 [02:04<34:37:59, 124.80s/it]

Raw Output for graph_0: <human>: Generate a graph with 50 nodes, 589 edges, an average degree of 23.5600, 3702 triangles, a clustering coefficient of 0.6226, a maximum k-core of 0, and 3 communities. Output the edge list only in the format [(source, target),...]. Ensure the graph respects the specified number of nodes, edges, and constraints. Do not provide explanations or additional text.
<assistant>: [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [0, 10], [0, 11], [0, 12], [0, 13], [0, 14], [0, 15], [0, 16], [0, 17], [0, 18], [0, 19], [0, 20], [0, 21], [0, 22], [0, 23], [0, 24], [0, 25], [0, 26], [0, 27], [0, 28], [0, 29], [0, 30], [0, 31], [0, 32], [0, 33], [0, 34], [0, 35], [0, 36], [0, 37], [0, 38], [0, 39], [0, 40], [0, 41], [0, 42], [0, 43], [0, 44], [0, 45], [0, 46], [0, 47], [0, 48], [0, 49], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [1, 9], [1, 10], [1, 11], [1, 12], [1, 13], [1, 14], [1, 15], [1, 16], [1, 17], [1, 18], [1, 19], [1, 20]

Processing prompts:   0%|          | 2/1000 [02:14<15:49:47, 57.10s/it] 

...

Raw Output for graph_109: <human>: Generate a graph with 24 nodes, 240 edges, an average degree of 20.0000, 1315 triangles, a clustering coefficient of 0.8619, a maximum k-core of 18, and 2 communities. Output the edge list only in the format [(source, target),...]. Ensure the graph respects the specified number of nodes, edges, and constraints. Do not provide explanations or additional text.
<assistant>: [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [0, 10], [0, 11], [0, 12], [0, 13], [0, 14], [0, 15], [0, 16], [0, 17], [0, 18], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [1, 9], [1, 10], [1, 11], [1, 12], [1, 13], [1, 14], [1, 15], [1, 16], [1, 17], [1, 18], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [2, 9], [2, 10], [2, 11], [2, 12], [2, 13], [2, 14], [2, 15], [2, 16], [2, 17], [2, 18], [3, 4], [3, 5], [3, 6], [3, 7], [3, 8], [3, 9], [3, 10], [3, 11], [3, 12], [3, 13], [3, 14], [3, 15], [3, 16], [3, 17], [3, 18], [4, 5], [4, 6], [4, 7], [4

Processing prompts:  11%|█         | 111/1000 [1:30:13<10:14:01, 41.44s/it]

Raw Output for graph_110: <human>: Generate a graph with 15 nodes, 60 edges, an average degree of 8.0000, 70 triangles, a clustering coefficient of 0.4918, a maximum k-core of 6, and 3 communities. Output the edge list only in the format [(source, target),...]. Ensure the graph respects the specified number of nodes, edges, and constraints. Do not provide explanations or additional text.
<assistant>: [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [0, 10], [0, 11], [0, 12], [0, 13], [0, 14], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [1, 9], [1, 10], [1, 11], [1, 12], [1, 13], [1, 14], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [2, 9], [2, 10], [2, 11], [2, 12], [2, 13], [2, 14], [3, 4], [3, 5], [3, 6], [3, 7], [3, 8], [3, 9], [3, 10], [3, 
Processing graph_111:
  Input tokens: 94
  Max new tokens: 748


Processing prompts:  11%|█         | 112/1000 [1:30:59<10:34:41, 42.88s/it]

Raw Output for graph_111: <human>: Generate a graph with 36 nodes, 216 edges, an average degree of 12.0000, 283 triangles, a clustering coefficient of 0.3460, a maximum k-core of 0, and 5 communities. Output the edge list only in the format [(source, target),...]. Ensure the graph respects the specified number of nodes, edges, and constraints. Do not provide explanations or additional text.
<assistant>: [[0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [0, 10], [0, 11], [0, 12], [0, 13], [0, 14], [0, 15], [0, 16], [0, 17], [0, 18], [0, 19], [0, 20], [0, 21], [0, 22], [0, 23], [0, 24], [0, 25], [0, 26], [0, 27], [0, 28], [0, 29], [0, 30], [0, 31], [0, 32], [0, 33], [0, 34], [0, 35], [4, 1], [4, 2], [4, 3], [4, 5], [4, 6], [4, 7], [4, 8], [4, 9], [4, 10], [4, 11], [4, 12], [4, 13], [4, 14], [4, 15], [4, 16], [4, 17], [4, 18], [4, 19], [4, 20], [4, 21], [4, 22], [4, 23], [4, 24], [4, 25], [4, 26], [4, 27], [4, 28], [4, 29], [4, 30], [4, 31], [4, 32], [4, 33], [4, 34], [4, 35], [5, 1], [5, 

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [12]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')